In [ ]:
import os
import cv2
import numpy as np
import json
import time
import logging
from tqdm import tqdm


"""
Copyright (c) Facebook, Inc. and its affiliates.
"""
import numpy as np
import os
import cv2
import json
import glob
from droidlet.lowlevel.robot_mover_utils import transform_pose
from numba import njit
from math import ceil, floor
import math

# Values for locobot in habitat. 
# TODO: generalize this for all robots
fx, fy = 256, 256
cx, cy = 256, 256
intrinsic_mat = np.array([[  fx, 0., cx],
                            [  0., fy, cy],
                            [  0., 0., 1.]])
# rotation from pyrobot to canonical coordinates (https://github.com/facebookresearch/fairo/blob/main/agents/locobot/coordinates.MD)
rot = np.array([[0.0, 0.0, 1.0], [-1.0, 0.0, 0.0], [0.0, -1.0, 0.0]])
CAMERA_HEIGHT = 0.6
trans = np.array([0, 0, CAMERA_HEIGHT])

 # TODO: Consolidate camera intrinsics and their associated utils across locobot and habitat.
def compute_uvone(height, width):
    intrinsic_mat_inv = np.linalg.inv(intrinsic_mat)
    img_resolution = (height, width)
    img_pixs = np.mgrid[0 : img_resolution[0] : 1, 0 : img_resolution[1] : 1]
    img_pixs = img_pixs.reshape(2, -1)
    img_pixs[[0, 1], :] = img_pixs[[1, 0], :]
    uv_one = np.concatenate((img_pixs, np.ones((1, img_pixs.shape[1]))))
    uv_one_in_cam = np.dot(intrinsic_mat_inv, uv_one)
    return uv_one_in_cam, intrinsic_mat, rot, trans

def convert_depth_to_pcd(depth, pose, uv_one_in_cam, rot, trans):
    # point cloud in camera frame
    depth = (depth.astype(np.float32) / 1000.0).reshape(-1)
    pts_in_cam = np.multiply(uv_one_in_cam, depth)
    pts_in_cam = np.concatenate((pts_in_cam, np.ones((1, pts_in_cam.shape[1]))), axis=0)
    # point cloud in robot base frame
    pts_in_base = pts_in_cam[:3, :].T
    pts_in_base = np.dot(pts_in_base, rot.T)
    pts_in_base = pts_in_base + trans.reshape(-1)
    # point cloud in world frame (pyrobot)
    pts_in_world = transform_pose(pts_in_base, pose)
    return pts_in_world

from scipy import ndimage 

def inspect(filtered, annot_img):
    # quantize to nearest neighbor values in annot_img
    before = np.sum(annot_img == 0)
    after = np.sum(filtered == 0)
    print(f'holes filed {(before-after)/before * 100} %')

    # do nearest neighbor quantization
            
    
    az = annot_img == 0
    # print(f'showing 
        
    # visualize side by side, and diff
    diff = (filtered != annot_img) & (annot_img == 0)
    # print(f'dtypes {annot_img.dtype, filtered.dtype}')
    # check some values
    
    a = filtered[diff == True].ravel()
    b = annot_img[diff == True].ravel()
    
    print(f'a {a.shape}, b {b.shape}, {a[:5], b[:5]}')
    
    # ndiff = a - b
    # print(f'diff histogram')
    # plt.hist(ndiff)
    # plt.show()
    
    vis_diff = np.zeros_like(annot_img)
    t = diff != 0
    # print(f'diff indices {t.shape, t[:3]}')
    
    vis_diff[diff != 0] = 1
    
    fig, axs = plt.subplots(1,3, figsize=(10,5))
    axs[0].imshow(annot_img)
    axs[1].imshow(filtered)
    axs[2].imshow(vis_diff)
    plt.show()
    
    # # what are the value distributions 
    # fig, axs = plt.subplots(1,2, figsize=(10,5))
    # axs[0].hist(annot_img)
    # axs[1].hist(filtered)
    # plt.show()
    
# def do_bilateral_filter(annot_img, 
from scipy.interpolate import griddata

def do_interp(annot_img, m):
    # print(f'annot_img.shape {annot_img.shape}')
    x = np.linspace(0,512,512).astype(np.uint16)
    y =  np.linspace(0,512,512).astype(np.uint16)
    # x = x[::-1]
    # print(y[::-1])
    
    X, Y = np.meshgrid(x,y)
    # print(X, Y)
    
    # (n,D) 
    # (n,)
    points = []
    for x in range(512):
        for y in range(512):
            points.append((x,y))
            
    points = np.asarray(points)
    # print(f'points.shape {points[:3]}')
    
    # print(f'annot_img.flatten().shape {annot_img.flatten().shape}')
    ze = np.argwhere(annot_img == 0).tolist()
    
    # visualize zeros
    zev = np.zeros_like(annot_img)
    for x, y in ze:
        zev[x][y] = 1
        
    plt.imshow(zev)
    plt.title(f'visualizing holes in the image')
    plt.show()
    
    # print(ze)
    # for x,y in ze:
    #     print(annot_img[max(x-4,0):][y])
    interp = griddata(points, annot_img.flatten(), ze, method=m)
    # plt.hist(interp)
    # plt.show()
    print(f'np.bincount {np.bincount(interp.astype(np.int32))}')
    print(f'interp.shape {interp.shape, interp[:3]}')
    
    b = np.copy(annot_img)
    
    for i in range(len(ze)):
        x,y = ze[i]
        prev, now = b[x][y], interp[i]
        if prev != now:
            print(f'prev {b[x][y]}, now {interp[i]}')
        b[x][y] = interp[i]
    return b

from collections import deque
    
# @njit
def get_annot(
    height, width, pts_in_cur_img, src_pts_in_cur_cam, cur_pts_in_cur_cam, src_label, valid_z, m, labels):
    """
    This creates the new semantic labels of the projected points in the current image frame. Each new semantic label is the 
    semantic label corresponding to pts_in_cur_img in src_label. 
    """
    annot_img = np.zeros((height, width)).astype(np.float32)
    for indx in range(len(pts_in_cur_img)):
        r = int(indx/width)
        c = int(indx - r*width)
        x, y, _ = pts_in_cur_img[indx]
        
        # We take ceil and floor combinations to fix quantization errors
        if floor(x) >= 0 and ceil(x) < height and floor(y) >=0 and ceil(y) < width and valid_z[indx]:
            cur_indx = ceil(x) + ceil(y)*width
            if src_pts_in_cur_cam[indx][2] - cur_pts_in_cur_cam[cur_indx][2] < 0.01:
                annot_img[ceil(y)][ceil(x)] = src_label[r][c]
                annot_img[floor(y)][floor(x)] = src_label[r][c]
                annot_img[ceil(y)][floor(x)] = src_label[r][c]
                annot_img[floor(y)][ceil(x)] = src_label[r][c]
      
    print(f'zeros {np.sum(annot_img == 0)}')
   
    
    #interpolate here
    if m == 'none':
        return annot_img
    
#     from scipy import ndimage
    dilated = ndimage.binary_dilation(annot_img).astype(annot_img.dtype)

    def closest_non_zero(a, x, y):
        
        def get_neighbors(a, curx, cury):
            ns = []
            if curx > 0:
                ns.append((curx-1, cury)) # n
            if cury > 0:
                ns.append((curx, cury-1)) # w
            if cury < 511:
                ns.append((curx, cury+1)) # e 
            if curx < 511:
                ns.append((curx+1, cury)) # s
#             if curx > 0 and cury > 0:
#                 ns.append((curx-1, cury-1)) # nw
#             if cury > 0:
#                 ns.append((curx1, cury-1)) # ne
#                 ns.append((curx+1, cury+1)) #se
#                 ns.append((curx+1, cury)) # sw
                
#             if curx < len(a)-1 and cury
            return ns

        bfsq = deque([])
        visited = np.zeros_like(a)
        bfsq.append((x,y))
        while True:
            curx, cury = bfsq.popleft()
            visited[curx][cury] = 1
            if a[curx][cury] > 0:
                return a[curx][cury]
            ns = get_neighbors(a, curx, cury)
            for n in ns:
                if visited[n] == 0:
                    bfsq.append(n)
        
        
        
    def do_nn_fill(annot_img):
        count = 0
        for x in range(len(annot_img)):
            for y in range(len(annot_img[0])):
                if annot_img[x][y] == 0:
                    annot_img[x][y] = closest_non_zero(annot_img, x, y)
        print(f'{count} holes')
        return annot_img
    
    # filled = do_nn_fill(annot_img)
        
    filtered = cv2.bilateralFilter(annot_img, 10, 50, 50)
    # filtered = np.round(filtered)
    
    def quantize(filtered, og):
        uq = np.unique(og.reshape(-1), axis=0)
        print(f'unique values {uq}')
        
        def find_nearest(array, value):
            array = np.asarray(array)
            idx = (np.abs(array - value)).argmin()
            return array[idx]
        
        qtized = 0
        for x in range(512):
            for y in range(512):
                n = find_nearest(uq, filtered[x][y])
                if n != filtered[x][y]:
                    qtized += 1
                filtered[x][y] = n
        print(f'{qtized} values quantized!')
        
        return filtered
    
    filtered = quantize(filtered, annot_img)
    
    inspect(filtered, annot_img)
    
    # return filtered

    # print(np.unique(dilated))
    
#     mask = functools.reduce(np.logical_or, (annot_img==l for l in labels)) 
#     annot_img = np.where(mask, annot_img, 0.)
    
#     print('masked image with specific labels only')
#     plt.imshow(annot_img)
#     plt.show()
    
    
    # b = do_interp(annot_img, m)
    # inspect(b, annot_img)
    
    # diff = interp - annot_img
    # print(f'plotting diff between interpolated and original ...')
    # plt.imshow(diff)
    # plt.show()
    
    return filtered

def visualize_mask(valid, height, width):
    annot_img = np.zeros((height, width))
    for indx in range(len(valid)):
        r = int(indx/width)
        c = int(indx - r*width)
        annot_img[r][c] = valid[indx]
        
    plt.imshow(annot_img)
    plt.title(f'visualize valid mask')
    plt.show()

import open3d as o3d

def visualize_pcd(xyz):
    
    # Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    o3d.visualization.draw_geometries([pcd])


def get_intersection(a, b):
    common = []
    for x in tqdm(a):
        for y in b:
            if np.allclose(x,y):
                common.append(x)
        if len(common) > 0:
            print(f'{len(common)} elements in common!')
    return common


class LabelPropagate:
    def __call__(self,    
        src_img,
        src_depth,
        src_label,
        src_pose,
        cur_img,
        cur_pose,
        cur_depth,
        interp,
        labels,
    ):
        """
        1. Gets point cloud for the source image 
        2. Transpose the point cloud based on robot location (cur_pose) 
        3. Project the point cloud back into the image frame. The corresponding semantic label for each point from the src_label becomes
        the new semantic label in the current frame.
        Args:
            src_img (np.ndarray): source image to propagte from
            src_depth (np.ndarray): source depth to propagte from
            src_label (np.ndarray): source semantic map to propagte from
            src_pose (np.ndarray): (x,y,theta) of the source image
            cur_pose (np.ndarray): (x,y,theta) of current image
            cur_depth (np.ndarray): current depth
        """

        height, width, _ = src_img.shape
        uv_one_in_cam, intrinsic_mat, rot, trans = compute_uvone(height, width)
        
        src_pts_in_world = convert_depth_to_pcd(src_depth, src_pose, uv_one_in_cam, rot, trans)
        
        # visualize using o3d
        # visualize_pcd(src_pts_in_world[-100000:])
        
        # convert pts_in_world to current base
        src_pts_in_cur_base = transform_pose(src_pts_in_world, (-cur_pose[0], -cur_pose[1], 0))
        src_pts_in_cur_base = transform_pose(src_pts_in_cur_base, (0.0, 0.0, -cur_pose[2]))
            
        # print(pts_in_cur_base.shape, pts_in_cur_base[:3], pts_in_cur_base[:,2][:3])
        # conver point from current base to current camera frame
        src_pts_in_cur_cam = src_pts_in_cur_base - trans.reshape(-1)
        src_pts_in_cur_cam = np.dot(src_pts_in_cur_cam, rot)
        
        # Get Valid Z
        valid_z = src_pts_in_cur_cam[:,2] > 0
        # print(f'valid_z {valid_z.shape, valid_z[:3]}')
        # print((valid_z == True).sum(), (valid_z == False).sum())
        
        # Filter based on current depth.
        cur_depth = (cur_depth.astype(np.float32) / 1000.0).reshape(-1)
        cur_pts_in_cur_cam = np.multiply(uv_one_in_cam, cur_depth).T 
        # cur_pts_in_world = convert_depth_to_pcd(cur_depth, cur_pose, uv_one_in_cam, rot, trans)
        # visualize_pcd(src_pts_in_cur_cam)
        # visualize_pcd(cur_pts_in_cur_cam)
        
        def multidim_intersect(arr1, arr2):
            arr1 = np.round(arr1, 4)
            arr2 = np.round(arr2, 4)
            arr1_view = arr1.T.view([('',arr1.dtype)]*arr1.shape[1]).T
            arr2_view = arr2.T.view([('',arr2.dtype)]*arr2.shape[1]).T
            print(arr1_view.shape, arr1_view[0], arr2_view.shape, arr2_view[:3])
            intersected = np.intersect1d(arr1_view, arr2_view)
            return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])
        
        
        print(f'src_pts_in_cur_cam.shape, cur_pts_in_cur_cam.shape {src_pts_in_cur_cam.dtype, cur_pts_in_cur_cam.dtype}')
            
        # common = multidim_intersect(src_pts_in_cur_cam, cur_pts_in_cur_cam)
        # print(f'{len(common)} elements in common!')
        
        # for the common points, indices won't be the same? 
        """
        for a point at x,y, I want to know the og depth and the current depth.
        I can get the og depth using the index (x,y).
        I can get the current depth using the index (x,y)
        """
        
        
        diff =  src_pts_in_cur_cam - cur_pts_in_cur_cam
        
        fig, axs = plt.subplots(1,3,figsize=(10,5))
        for i in range(3):  
            axs[i].imshow(diff[:,i].reshape((512,512)))
            axs[i].set_title(f'axis {i}')
        plt.show()
        
        diff_d = diff[:,2] # np.linalg.norm(diff, axis=1)
        # print(f'diff_d {diff_d.shape}')
        # plt.hist(diff_d)
        # plt.show()
        
#         print(f'pts_in_cur_cam.shape {src_pts_in_cur_cam.shape}, cur_pts_in_cam.shape {cur_pts_in_cur_cam.shape}')
        
#         # print(np.unique(
#         print(f'diff.min() {diff_d.min()}, diff.max() {diff_d.max()}, diff.shape {diff_d.shape}')
#         depth_thresh = 0.01
#         t = np.where(diff_d < depth_thresh, True, False)
        
#         visualize_mask(t, height, width)
        
        # print(f'np.bincount {np.bincount(t)}')
        # plt.imshow(diff_d.reshape((512,512)))
        # plt.title(f'visualize l2 norms, diff_d')
        # plt.show()
        
        # print(t.shape, t[:3])
        # valid_z = np.logical_and(valid_z, t)    
        # visualize_pcd(src_pts_in_cur_cam[valid_z])
        # print(f'valid_z.shape {valid_z.shape}')
        
        
        # conver pts in current camera frame into 2D pix values
        src_pts_in_cur_img = np.matmul(intrinsic_mat, src_pts_in_cur_cam.T).T
        # print(pts_in_cur_img.shape, pts_in_cur_img[:5])
        
        src_pts_in_cur_img /= src_pts_in_cur_img[:, 2].reshape([-1, 1])
        
        # take a mask of all valid indices
        valid_idx = np.logical_and(
            np.logical_and(0 <= src_pts_in_cur_img[:, 0], src_pts_in_cur_img[:, 0] < height),
            np.logical_and(0 <= src_pts_in_cur_img[:, 1], src_pts_in_cur_img[:, 1] < width),
        ).reshape((512,512))
        
        
        
        
        fig, axs = plt.subplots(1,3,figsize=(16,8))
        axs[0].imshow(src_img)
        axs[0].set_title('og image')
        axs[1].imshow(cur_img)
        axs[1].set_title('cur image')
        
        vis_c = np.array(src_img, copy=True)
        vis_c[valid_idx == False] = (0,0,0)
        
        axs[2].imshow(vis_c)
        axs[2].set_title('part of og image still visible')
        plt.show()
        
        # print(cur_pts_in_world[:4], pts_in_world[:4])
        
        # print(
        # valid = pts_in_cur_img
        # print(pts_in_cur_img[:5])
        # print(pts_in_cur_img.min(), pts_in_cur_img.max())
        
        return get_annot(height, width, src_pts_in_cur_img, src_pts_in_cur_cam, cur_pts_in_cur_cam, src_label, valid_z, interp, labels)

# LABEL_PROP_TEST_ASSETS_DIR = '/private/home/apratik/fairo/perception_handlers/label_prop_test_assets'
# LABEL_PROP_OCCLUSION_DATA = '/checkpoint/apratik/jobs/reexplore/largerun1/baselinev3/2/instance/5/0/r1/'
LABEL_PROP_OCCLUSION_DATA = '/home/locobotm/Downloads/r2'

import matplotlib.pyplot as plt 
import functools

class LabelPropTest:
    def __init__(self):
        self.lp = LabelPropagate()
        self.test_assets = LABEL_PROP_OCCLUSION_DATA

    def read_test_asset_idx(self, root, img_indx):
        src_img = cv2.imread(os.path.join(root, "rgb/{:05d}.jpg".format(img_indx)))
        src_depth = np.load(os.path.join(root, "depth/{:05d}.npy".format(img_indx)))
        src_label = np.load(os.path.join(root, "seg/{:05d}.npy".format(img_indx)))
        # print(f'root {root}')
        # print(f'rgb/{img_indx:05d}.jpg, depth/{img_indx:05d}.npy, seg/{img_indx:05d}.npy')
        with open(os.path.join(root, "data.json"), "r") as f:
            base_pose_data = json.load(f)
        src_pose = base_pose_data["{}".format(img_indx)]
        
        # Visualize label
        return src_img, src_label, src_depth, src_pose, None
    
    def calculate_accuracy(self, act, pred, labels):
        h, w = act.shape
        assert act.shape == pred.shape
        
        mask = functools.reduce(np.logical_or, (pred==l for l in labels)) 
        masked = np.where(mask, pred, 0.)
        # plt.imshow(masked)
        # plt.show()

        correct = np.sum(act[mask] == pred[mask])
        total = np.sum(pred[mask] != 0)
        
        return correct/total
    
    def calculate_accuracy_v0(self, act, pred):
        assert act.shape == pred.shape    
        correct = np.sum(act[pred != 0] == pred[pred != 0])
        total = np.sum(pred != 0)
        return correct/total
    
    def _run_test(self, data_dir, interp):
        """
        Checks that each label prop call runs in < 0.1 seconds with > 90% accuracy
        """
        # for x in os.listdir(data_dir)[-1]:
        dd = data_dir

        # Each test asset folder has one source id, and one id to label propagate to
        ps = range(1,18,6)
        
        # with open(os.path.join(dd, 'gtids.txt'), 'r') as f:
        #     ids = f.readlines()
        #     ids = [int(x.strip()) for x in ids]

        src_img, src_label, src_depth, src_pose, cam_transform = self.read_test_asset_idx(dd, 0)
        
        accuracies, accuracies_v0 = [], []
        
        labels_chosen = [404,243, 133]
        
        for i in tqdm(range(len(ps))):
            p = ps[i]
            print(f'p {p}')
            cur_img, cur_label, cur_depth, cur_pose, cam_transform = self.read_test_asset_idx(dd, p)

            # src_depth = np.zeros_like(src_depth)
            # src_depth[100,100] = 2

            start = time.time()
            prop_label = self.lp(
                src_img, src_depth, src_label, src_pose, cur_img, cur_pose, cur_depth, interp, labels_chosen
            )
            
            filtered = np.zeros_like(prop_label)
            filtered[prop_label == 133] = 133
            # print(f'filtered non_zero {filtered[filtered > 0].sum()}')
        
            time_taken = time.time() - start
            acc = self.calculate_accuracy(cur_label, prop_label, labels_chosen)
            acc_v0 = self.calculate_accuracy_v0(cur_label, prop_label)
            accuracies.append(round(acc,2))
            accuracies_v0.append(round(acc_v0,2))
            
            # print(f'time {time_taken}, acc {acc}')
            
            if True: #i == 11:
                fig, axs = plt.subplots(1,3,figsize=(16,8))
                axs[0].imshow(cur_label)
                axs[0].set_title('GT Label')
                axs[1].imshow(filtered)
                axs[1].set_title(f'Propagated p{p}')
                axs[2].imshow(prop_label)
                axs[2].set_title(f'Propagated p{p}')
                plt.savefig("bilateral_filtering_v1/{:04d}.jpg".format(i))
                plt.show()

        print(f'plotting accuracies for interpolation {interp}')
        plt.plot(range(len(accuracies)), accuracies, label='chosen labels')
        plt.plot(range(len(accuracies)), accuracies_v0, label='all labels')
        plt.legend(loc="upper right")
        for i, acc in enumerate(accuracies):
            plt.annotate(acc, (i, accuracies[i]))
        for i, acc in enumerate(accuracies_v0):
            plt.annotate(acc, (i, accuracies_v0[i]))
        plt.savefig("accuracy.jpg")
        plt.show()
            
#         for x in range(70):
#             # yaw = x/20
#             delta += 0.1
#             cur_pose[2] += 0.1

#             print(f'src_pose {src_pose[2]} cur_pose {cur_pose[2]}, delta {math.degrees(cur_pose[2])}')
#             start = time.time()
#             prop_label = self.lp(src_img, src_depth, src_label, src_pose, cur_pose, cur_depth)

#             fig, axs = plt.subplots(1,2,figsize=(10,5))
#             # axs[0].imshow(src_img)
#             axs[0].imshow(src_label)
#             axs[1].set_title('Origin')
#             # axs[2].imshow(cur_img)
#             axs[1].imshow(prop_label)
#             axs[1].set_title(f'delta in degrees {math.degrees(delta)}')
#             # plt.show()
#             plt.savefig("rot/{:04d}.jpg".format(ctr))
#             plt.show()
#             ctr += 1

#             time_taken = time.time() - start
#             acc = self.calculate_accuracy(cur_label, prop_label)
#             print(f'time {time_taken}, acc {acc}')
#             # assert acc*100 > 90, f'accuracy {acc} < 90'
        
    def test_label_prop_nonoise(self, interp):
        data_dir = os.path.join(self.test_assets)
        self._run_test(data_dir, interp)
            
    # def test_label_prop_noise(self):
    #     data_dir = os.path.join(self.test_assets, 'noise')
    #     self._run_test(data_dir)
        
c = LabelPropTest()

# x = np.zeros((512, 512)).astype(np.int32)
# x[:100,:100] = True
# print(np.bincount(x.flatten()))
# plt.imshow(x)
# plt.show()

for interp in ['nearest']:
    c.test_label_prop_nonoise(interp)